Example usage of FlowClient to run daily location, modal location and flow queries.

In [ ]:
import flowclient

## Create an access token
[Generate an API access token](../index.md#flowauth) using FlowAuth.

## Create an API connection
Paste the access token from FlowAuth here as `TOKEN`

In [ ]:
conn = flowclient.connect(
    url=os.getenv("FLOWAPI_URL", "http://localhost:9090"),
    token=TOKEN,
    ssl_certificate=os.getenv("SSL_CERTIFICATE_FILE"),
)

## Daily location

### Specify query parameters

In [ ]:
query_spec = flowclient.aggregates.spatial_aggregate_spec(
    locations=flowclient.daily_location_spec(
        date="2016-01-01",
        aggregation_unit="admin3",
        method="last",
        subscriber_subset=None,
    )
)

query_spec

### Set query running

In [ ]:
query_id = flowclient.run_query(connection=conn, query_spec=query_spec)
query_id

### Check status of query

In [ ]:
flowclient.get_status(connection=conn, query_id=query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

Alternative (run query, wait for it to finish, and get result, in one function call):

In [ ]:
df = flowclient.get_result(connection=conn, query_spec=query_spec)
df.head()

## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [ ]:
dl1 = flowclient.daily_location_spec(
    date="2016-01-01",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)
dl2 = flowclient.daily_location_spec(
    date="2016-01-02",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)
ml = flowclient.modal_location_spec(locations=[dl1, dl2])

ml

Or from a date range:

In [ ]:
ml = flowclient.modal_location_from_dates_spec(
    start_date="2016-01-01",
    end_date="2016-01-03",
    aggregation_unit="admin3",
    method="last",
    subscriber_subset=None,
)

ml

### Set query running

In [ ]:
query_id = flowclient.run_query(
    connection=conn,
    query_spec=flowclient.aggregates.spatial_aggregate_spec(locations=ml),
)
query_id

### Check status of query

In [ ]:
flowclient.get_status(connection=conn, query_id=query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

Alternative:

In [ ]:
df = flowclient.get_result(
    connection=conn,
    query_spec=flowclient.aggregates.spatial_aggregate_spec(locations=ml),
)
df.head()

## Flow
### Specify query parameters
Specify parameters for two location queries:

In [ ]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location_spec(
    date="2016-01-01",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location_spec(
    date="2016-01-07",
    aggregation_unit=aggregation_unit,
    method="last",
    subscriber_subset=None,
)

Specify parameters for flow query:

In [ ]:
flow = flowclient.aggregates.flows_spec(
    from_location=dl_past,
    to_location=dl_present,
)
flow

### Set query running

In [ ]:
query_id = flowclient.run_query(connection=conn, query_spec=flow)
query_id

### Check status of query

In [ ]:
flowclient.get_status(connection=conn, query_id=query_id)

### Get query result

In [ ]:
df = flowclient.get_result_by_query_id(connection=conn, query_id=query_id)
df.head()

## Counts of Events by Location

In [ ]:
df = flowclient.get_result(
    connection=conn,
    query_spec=flowclient.aggregates.location_event_counts_spec(
        start_date="2016-01-01",
        end_date="2016-01-08",
        aggregation_unit="admin3",
        count_interval="hour",
    ),
)
df.head()